# Section 1-1 - Basic Neural Network - Titanic

In the previous section, we simply iterated through randomly-generated matrices and chose the best-performing one. We build on this approach by reducing loss in a systematic way via stochastic gradient descent. In particular, we'll be using TensorFlow, an open source library developed by Google, and Keras, a high-level wrapper on top of TensorFlow.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('data/titanic.csv')

In [2]:
df_train = df.iloc[:712, :]

scaler = StandardScaler()
features = ['Class', 'Sex', 'Age', 'Fare']

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [3]:
df_test = df.iloc[712:, :]

X_test = scaler.transform(df_test[features].values)
y_test = df_test['Survived'].values

## Benchmark

In [4]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['Survived'].values)

y_prediction = model.predict(X_test)
print("\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


## 1-layer Neural Network

Instead of generating a linear stack of layers with Numpy, we'll be implementing our model using Keras. We initialize our model, add a layer that inputs vectors of length 4 and outputs vectors of length 2, and finally add a softmax layer. We configure the learning process in the compilation step by specifying the optimizer, loss function and performance metric.

Stochastic gradient descent acts by changing the weights gradually in the 'direction' that decreases the average loss. In other words, a particular weight would be increased if acts to decrease loss, or the weight decreased if it acts to increase loss. TensorFlow does the heavy-lifting by efficiently handling these numerical computations under the hood. A simple example of stochastic gradient descent is illustrated in the Appendix.

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot, epochs=12)

model.summary()

print('\ntime taken %s seconds' % str(time() - start))

2023-03-24 10:35:29.824060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/12
23/23 [==============================] - 0s 884us/step - loss: 1.4785 - accuracy: 0.2640
Epoch 2/12
23/23 [==============================] - 0s 828us/step - loss: 1.3076 - accuracy: 0.3006
Epoch 3/12
23/23 [==============================] - 0s 764us/step - loss: 1.1619 - accuracy: 0.3329
Epoch 4/12
23/23 [==============================] - 0s 841us/step - loss: 1.0360 - accuracy: 0.3624
Epoch 5/12
23/23 [==============================] - 0s 835us/step - loss: 0.9274 - accuracy: 0.4396
Epoch 6/12
23/23 [==============================] - 0s 797us/step - loss: 0.8380 - accuracy: 0.5183
Epoch 7/12
23/23 [==============================] - 0s 935us/step - loss: 0.7617 - accuracy: 0.5674
Epoch 8/12
23/23 [==============================] - 0s 873us/step - loss: 0.7028 - accuracy: 0.6110
Epoch 9/12
23/23 [==============================] - 0s 869us/step - loss: 0.6543 - accuracy: 0.6531
Epoch 10/12
23/23 [==============================] - 0s 848us/step - loss: 0.6172 - accuracy: 0.6910

In [6]:
y_prediction = np.argmax(model.predict(X_test), axis=-1)
print("\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

6/6 [==============================] - 0s 870us/step

accuracy 0.776536312849162


We notice that the loss reduces systematically as the model 'learns' from the data. The rate of loss reduction, however, seems to indicate that loss could be further reduced.

## 2-layer Neural Network

In [7]:
start = time()

model = Sequential()
model.add(Dense(100))
model.add(Dense(2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot, epochs=12)

model.summary()

print('\ntime taken %s seconds' % str(time() - start))

Epoch 1/12
23/23 [==============================] - 0s 921us/step - loss: 0.6226 - accuracy: 0.7388
Epoch 2/12
23/23 [==============================] - 0s 914us/step - loss: 0.5424 - accuracy: 0.7823
Epoch 3/12
23/23 [==============================] - 0s 910us/step - loss: 0.5081 - accuracy: 0.7795
Epoch 4/12
23/23 [==============================] - 0s 935us/step - loss: 0.4903 - accuracy: 0.7767
Epoch 5/12
23/23 [==============================] - 0s 925us/step - loss: 0.4817 - accuracy: 0.7753
Epoch 6/12
23/23 [==============================] - 0s 945us/step - loss: 0.4760 - accuracy: 0.7767
Epoch 7/12
23/23 [==============================] - 0s 951us/step - loss: 0.4731 - accuracy: 0.7767
Epoch 8/12
23/23 [==============================] - 0s 955us/step - loss: 0.4709 - accuracy: 0.7781
Epoch 9/12
23/23 [==============================] - 0s 955us/step - loss: 0.4690 - accuracy: 0.7781
Epoch 10/12
23/23 [==============================] - 0s 912us/step - loss: 0.4678 - accuracy: 0.7781

In [8]:
y_prediction = np.argmax(model.predict(X_test), axis=-1)
print("\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

6/6 [==============================] - 0s 913us/step

accuracy 0.8100558659217877


The loss reduction 'flattens out' more compared to the 1-layer example, and the accuracy improves to 80%.

## 3-layer Neural Network

In [9]:
start = time()

model = Sequential()
model.add(Dense(100))
model.add(Dense(100))
model.add(Dense(2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot, epochs=12)

model.summary()

print('\ntime taken %s seconds' % str(time() - start))

Epoch 1/12
23/23 [==============================] - 0s 1ms/step - loss: 0.5475 - accuracy: 0.7346
Epoch 2/12
23/23 [==============================] - 0s 989us/step - loss: 0.4941 - accuracy: 0.7725
Epoch 3/12
23/23 [==============================] - 0s 1ms/step - loss: 0.4781 - accuracy: 0.7725
Epoch 4/12
23/23 [==============================] - 0s 946us/step - loss: 0.4724 - accuracy: 0.7767
Epoch 5/12
23/23 [==============================] - 0s 951us/step - loss: 0.4693 - accuracy: 0.7767
Epoch 6/12
23/23 [==============================] - 0s 946us/step - loss: 0.4675 - accuracy: 0.7795
Epoch 7/12
23/23 [==============================] - 0s 952us/step - loss: 0.4660 - accuracy: 0.7809
Epoch 8/12
23/23 [==============================] - 0s 1ms/step - loss: 0.4653 - accuracy: 0.7851
Epoch 9/12
23/23 [==============================] - 0s 965us/step - loss: 0.4654 - accuracy: 0.7809
Epoch 10/12
23/23 [==============================] - 0s 1ms/step - loss: 0.4650 - accuracy: 0.7837
Epoch 1

In [10]:
y_prediction = np.argmax(model.predict(X_test), axis=-1)
print("\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

6/6 [==============================] - 0s 968us/step

accuracy 0.8044692737430168


We're not able to reduce loss on the training that much further; the best performance obtained does not beat the benchmark. Since the dataset is small, there isn't as much for the model to 'learn' from (or for that matter, predict on). We'll apply techniques developed so far on a much larger dataset in the next section.